In [1]:
# useing this variable for saving models and summaries
exp_name = "DWN_Wmultitask_8_opp"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../../src')

In [3]:
import models.wavelet_conv_model as wavelet_conv_model
import training
import evaluation.nn_eval_multitask as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../../data/opp8.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)

In [5]:
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

In [6]:
train_data.shape, train_gt.shape

((691, 1024, 77), (691, 1024, 19))

In [7]:
test_data.shape, test_gt.shape

((119, 1024, 77), (119, 1024, 19))

## Model

In [8]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 100

wavelet_levels = 8    #number of wavelet layer to have in the network
convolution_levels = 3    #number of convolution to have in the network
pooling_stride = [2, 2, 2]
weights_shape = [[5, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32]
wavelet_weights_shape = [[5, n_variables, n_variables]]    #high_pass and low_pass dimensions
activation = tf.nn.relu
n_classes = 19

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)
pooled_gt = np.reshape(train_gt[:,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)

variables_weights = np.ones((n_classes,))

In [9]:
train_data.shape

(691, 1024, 77)

In [10]:
pos_weight.shape

(19,)

In [11]:
pos_weight

array([  0.40793684,  51.83632019,  50.54312354,  53.39606396,
        53.49661121,  43.82919412,  45.64978903,  64.85852569,
        69.47649402,  84.21001927,  98.26823793,  89.34525026,
        99.39500568,  75.77777778,  76.17975567,  46.17226667,
        13.60019809,  65.60240964,  20.4471387 ])

In [12]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [13]:
summary_path = '../../summary/' + exp_name 
checkpoint_path = '../../model/' + exp_name + '/checkpoint' 

In [14]:
model = wavelet_conv_model.Model(n_variables = n_variables,
                                 learning_rate = learning_rate,
                                 n_classes = n_classes)

In [15]:
model.build_neural_wavelet_layer(pos_weight = pos_weight, 
                                 learning_rate = learning_rate,
                                 wavelet_levels = wavelet_levels,    
                                 convolution_levels = convolution_levels,    
                                 pooling_stride = pooling_stride,     
                                 weights_shape = weights_shape,
                                 bias_shape = bias_shape,
                                 wavelet_weights_shape = wavelet_weights_shape,    
                                 activation = activation,
                                 trainable_wavelet = True,
                                 trainable_conv = True,
                                 trainable_last = True,
                                 variables_weights = variables_weights)

In [16]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'multitask')

In [17]:
trainer.train(max_iter = 20000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.062 vs. 0.059
F1 = 0.097 vs. 0.087
Accuracy = 0.015 vs. 0.056
CHANGE DETECTION
AUC = 0.036 vs. 0.048
F1 = 0.093 vs. 0.092
Precision = 0.049 vs. 0.049
Recall = 0.948 vs. 0.698
Validation
ACTIVITY RECOGNITION
AUC = 0.056 vs. 0.056
F1 = 0.079 vs. 0.074
Accuracy = 0.015 vs. 0.056
CHANGE DETECTION
AUC = 0.019 vs. 0.025
F1 = 0.054 vs. 0.051
Precision = 0.028 vs. 0.026
Recall = 0.948 vs. 0.874
Minibatch Loss= 1.378117
Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.504 vs. 0.058
F1 = 0.542 vs. 0.084
Accuracy = 0.487 vs. 0.056
CHANGE DETECTION
AUC = 0.101 vs. 0.046
F1 = 0.174 vs. 0.093
Precision = 0.107 vs. 0.049
Recall = 0.458 vs. 0.907
Validation
ACTIVITY RECOGNITION
AUC = 0.282 vs. 0.057
F1 = 0.364 vs. 0.075
Accuracy = 0.452 vs. 0.056
CHANGE DETECTION
AUC = 0.061 vs. 0.026
F1 = 0.122 vs. 0.051
Precision = 0.082 vs. 0.026
Recall = 0.242 vs. 0.969
Minibatch Loss= 0.438436
Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.642 vs. 0.059


In [18]:
trainer.train(max_iter = 5000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.997 vs. 0.058
F1 = 0.990 vs. 0.086
Accuracy = 0.983 vs. 0.056
CHANGE DETECTION
AUC = 0.785 vs. 0.053
F1 = 0.838 vs. 0.095
Precision = 0.733 vs. 0.050
Recall = 0.978 vs. 0.858
Validation
ACTIVITY RECOGNITION
AUC = 0.613 vs. 0.057
F1 = 0.635 vs. 0.076
Accuracy = 0.894 vs. 0.056
CHANGE DETECTION
AUC = 0.122 vs. 0.025
F1 = 0.210 vs. 0.050
Precision = 0.133 vs. 0.026
Recall = 0.510 vs. 0.863
Minibatch Loss= 0.014846
Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.998 vs. 0.057
F1 = 0.993 vs. 0.082
Accuracy = 0.988 vs. 0.056
CHANGE DETECTION
AUC = 0.717 vs. 0.044
F1 = 0.834 vs. 0.089
Precision = 0.738 vs. 0.047
Recall = 0.960 vs. 0.845
Validation
ACTIVITY RECOGNITION
AUC = 0.607 vs. 0.057
F1 = 0.635 vs. 0.074
Accuracy = 0.894 vs. 0.056
CHANGE DETECTION
AUC = 0.123 vs. 0.025
F1 = 0.212 vs. 0.052
Precision = 0.147 vs. 0.027
Recall = 0.379 vs. 0.724
Minibatch Loss= 0.013525
Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.997 vs. 0.058


In [20]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../../model/DWN_Wmultitask_8_opp/checkpoint-25000'

In [21]:
#load the model
#saver = tf.train.Saver()
#saver.restore(sess, "../model" + "/CNN7002_Gleam" + "/CNN7002_Gleam")
#saver.restore(sess, '../model/DWN_activity_multitask_64_opp/checkpoint-20000')


In [22]:
#This is the model evaluation on test dataset
result = nn_eval.evaluate_model (model, sess, test_data, test_gt)

ACTIVITY RECOGNITION
AUC = 0.640 vs. 0.056
F1 = 0.638 vs. 0.074
Accuracy = 0.895 vs. 0.056
CHANGE DETECTION
AUC = 0.119 vs. 0.024
F1 = 0.208 vs. 0.050
Precision = 0.132 vs. 0.026
Recall = 0.490 vs. 0.660


In [23]:
out = nn_eval.deploy(model, sess, test_data, test_gt)

In [24]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
#probability_change = np.squeeze(out, axis = 2)

In [25]:
savemat('../../poc/poc'+exp_name+'.mat', {'poc':out[:,:,-1]})

In [ ]:
out.shape